### Análisis de Dataset LEGO
La práctica final de nuestro grupo constará del análisis del dataset publicado por Rachel Tatman, compilado por Rebrickable sobre información de piezas, sets, contenido de sets y colores de piezas de todos los sets originales de LEGO (acualizado hasta Julio de 2017).

Link al dataset:
https://www.kaggle.com/datasets/rtatman/lego-database

Este parece extraido de un esquema de base de datos relacional.

### Análisis Exploratorio

Para comenzar con la práctica, comenzaremos por importar el dataset y aplicar diferentes técnicas explorativas vistas para comprender mejor con qué información contamos, y la relación entre las diferentes características contempladas.

Luego, decidiremos cuál de las preguntas propuestas elegiremos para basar el resto de la práctica integradora.

Estas preguntas pueden ser:

 - ¿Cómo evolucionaron los sets de lego en tamaño a través de los años?
 - ¿Existe alguna asociación entre los colores y las temáticas?
 - ¿Podría predecir a que temática pertenece un set basado en el 
contenido de este? (recomendado)
 - A través de los años, ¿Cuál o cuáles son los sets que tienen las piezas 
más raras?
 - ¿Cómo evolucionaron los colores en los sets de lego a través de los 
años?


## Sets de Datos

Antes de importar el dataset, podemos bservar dentro de la carpeta /data que éste se encuentra distribuido en diferentes archivos CSV.
Dentro de la carpeta también se encuentra un DER que explica cómo estos datos se asemblan. Describamos por encima el propósito de cada archivo/tabla:

| Archivo/tabla       | Descripción |
| --------------------|-------------|
| colors.csv          | Id, nombre, valores RGB del color de la pieza e indicativo de si es transparente             |             |
| inventories.csv     | Asocia conjuntos de piezas individuales (de determinado color y características, u otros subsets, a un determinado set.            |
| inventory_parts.csv | Asocia un color, cantidad de piezas, y un booleano de repuesto entre un inventario y un tipo de pieza           |
| inventory_sets.csv  | Asocia un set en particular a un inventario (subsets como parte del inventario de otro set).            |
| part_categories.csv | Etiquetas de clasificación de los tipos de pieza LEGO (un Enum)            |
| parts.csv           | Tipos de piezas LEGO, con nombre y una categoría            |
| sets.csv            | Productos LEGO vendidos en sí, son paquetes que incluyen piezas de diversos tipos y subsets. Se guardan diferentes datos como año de lanzamiento, nombre, etc.           |
| themes.csv          | Etiqueta de clasificación de los diferentes sets.            |




In [7]:
# Importamos librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={
    'axes.facecolor': 'black',
    'figure.facecolor': 'black',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'axes.titlecolor': 'white'
})


In [18]:
# Cargamos el dataset
colors = pd.read_csv("./data/colors.csv")
inventories = pd.read_csv("./data/inventories.csv")
inventory_parts = pd.read_csv("./data/inventory_parts.csv")
inventory_sets = pd.read_csv("./data/inventory_sets.csv")
part_categories = pd.read_csv("./data/part_categories.csv")
parts = pd.read_csv("./data/parts.csv")
sets = pd.read_csv("./data/sets.csv")
themes = pd.read_csv("./data/themes.csv")


# Visualizamos las primeras filas del dataset
display(colors.head())
display(inventories.head())
display('\n')
display(inventory_parts.head())
display(inventory_sets.head())
display(part_categories.head())
display(parts.head())
display(sets.head())

,id,name,rgb,is_trans
0,-1,Unknown,0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f
3,2,Green,237841,f
4,3,Dark Turquoise,008F9B,f


,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1
3,15,1,5158-1
4,16,1,903-1


'\n'

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f
3,1,paddle,0,1,f
4,3,11816pr0005,78,1,f


,inventory_id,set_num,quantity
0,35,75911-1,1
1,35,75912-1,1
2,39,75048-1,1
3,39,75053-1,1
4,50,4515-1,1


,id,name
0,1,Baseplates
1,2,Bricks Printed
2,3,Bricks Sloped
3,4,"Duplo, Quatro and Primo"
4,5,Bricks Special


,part_num,name,part_cat_id
0,0687b1,Set 0687 Activity Booklet 1,17
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1


,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


Viendo las primeras filas de cada dataframe, observamos que estos datos son de carácter relacional (seguramente obtenidos en una base de datos estructurada, por sus llaves primarias y foráneas).

Es por eso que habrá que realizar merging/joining respecto a las preguntas que deseamos responder.

También podemos ver que hay datos que probablemente necesiten ser re-estructurados para servir en propósito comparativo, como lo es el tamaño de las piezas (sus dimensiones no están almacenadas como números).

Un análisis estadístico panorámico podría no ser tan relevante en este caso, ya que la gran mayoría de valores se tratan de identificadores y categorías. Pasaremos a definir qué variables tenemos.

### Análisis y clasificación de variables

Con un simple vistazo y sin contemplar la calidad y consistencia de las características del dataset, podemos realizar una pequeña descripción y clasificación de las variables que presenta: